In [218]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

"""from pytorch_pretrained_bert import BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
    "bert-base-uncased",
)

file_train = "trainset.xlsx"
datafile = file_train
train = pd.read_excel(datafile)


file_test = "testsetBERT.xlsx"
datafile_test = file_test
test = pd.read_excel(datafile_test)"""



'from pytorch_pretrained_bert import BertTokenizer\nbert_tok = BertTokenizer.from_pretrained(\n    "bert-base-uncased",\n)\n\nfile_train = "trainset.xlsx"\ndatafile = file_train\ntrain = pd.read_excel(datafile)\n\n\nfile_test = "testsetBERT.xlsx"\ndatafile_test = file_test\ntest = pd.read_excel(datafile_test)'

In [219]:
import torch

file_train = "trainset.xlsx"
datafile = file_train
train = pd.read_excel(datafile)

train['text'] = train['text'].str.replace("[^a-zA-Z]", " ")

import nltk
nltk.download('stopwords')


from nltk.corpus import stopwords 
stop_words = stopwords.words('english')
tokenized_doc = train['text'].apply(lambda x: x.split())



# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(train)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 
    
train['text'] = detokenized_doc 

import random
num = random.randrange(0,10)
train['text'] = ['[CLS] '+ text + ' [SEP]' for text in train['text']]
#splittext = train['text'][1000].split()

train['text'] = [token.split() for token in train['text']]

train['orig'] = np.nan
train['index_for_orig'] = np.nan
train['pred_word'] = np.nan


train['text'].sample()

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/osrivastava/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1373    [[CLS], uk, economy, facing, major, risks, uk,...
Name: text, dtype: object

In [220]:
index_val = 0
for split_token in train['text']:
    size = min(len(split_token),512)
    index_text = int(random.randrange(3,size-1))
    orig_word = split_token[index_text]
    train['text'][index_val][index_text] = '[MASK]'
    train['orig'][index_val] = orig_word
    train['index_for_orig'][index_val] = index_text  
    index_val = index_val + 1

/home/osrivastava/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/osrivastava/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [221]:
# de-tokenization 
detokenized_doc = [] 
for i in range(len(train)): 
    t = ' '.join(train['text'][i]) 
    detokenized_doc.append(t) 
    
train['text'] = detokenized_doc

In [222]:
train['text'][0]

'[CLS] tv future hands viewers home theatre systems plasma high definition tvs digital video recorders moving living room way people watch tv radically different five years time according expert panel gathered annual consumer electronics show las vegas discuss new technologies impact one favourite pastimes us leading trend programmes content delivered viewers via home networks cable satellite telecoms companies broadband service providers front rooms portable devices one talked technologies ces digital personal video recorders dvr pvr set top boxes like us tivo uk sky system allow people record store play pause forward wind tv programmes want essentially technology allows much personalised tv also built high definition tv sets big business japan us slower take europe lack high definition programming people forward wind adverts also forget abiding network channel schedules putting together la carte entertainment us networks cable satellite companies worried means terms advertising reven

In [223]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
bert_model_mlm = BertForMaskedLM.from_pretrained("bert-base-uncased")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model_mlm.eval()

index_val = 0

for text in train['text']:
    tokenized_text = tokenizer.tokenize(text)
    if len(tokenized_text) > 512:
        tokenized_text = tokenized_text[:512]
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    indic = int(train['index_for_orig'][index_val])
    print("Hi ", index_val, indic)
    # Predict all tokens
    with torch.no_grad():
        predictions = bert_model_mlm(tokens_tensor, segments_tensors)

    predicted_index = torch.argmax(predictions[0, indic]).item()
    train['pred_word'][index_val] = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    index_val = index_val + 1

Hi  0 253


/home/osrivastava/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Hi  1 12
Hi  2 82
Hi  3 150
Hi  4 71
Hi  5 330
Hi  6 100
Hi  7 22
Hi  8 7
Hi  9 77
Hi  10 157
Hi  11 100
Hi  12 159
Hi  13 146
Hi  14 11
Hi  15 5
Hi  16 212
Hi  17 15
Hi  18 90
Hi  19 87
Hi  20 111
Hi  21 59
Hi  22 289
Hi  23 48
Hi  24 219
Hi  25 29
Hi  26 67
Hi  27 112
Hi  28 134
Hi  29 98
Hi  30 76
Hi  31 123
Hi  32 53
Hi  33 93
Hi  34 77
Hi  35 59
Hi  36 17
Hi  37 40
Hi  38 212
Hi  39 114
Hi  40 24
Hi  41 7
Hi  42 66
Hi  43 8
Hi  44 178
Hi  45 377
Hi  46 237
Hi  47 111
Hi  48 3
Hi  49 98
Hi  50 101
Hi  51 26
Hi  52 167
Hi  53 92
Hi  54 30
Hi  55 72
Hi  56 174
Hi  57 123
Hi  58 115
Hi  59 106
Hi  60 216
Hi  61 152
Hi  62 111
Hi  63 44
Hi  64 15
Hi  65 196
Hi  66 17
Hi  67 9
Hi  68 120
Hi  69 73
Hi  70 265
Hi  71 106
Hi  72 145
Hi  73 112
Hi  74 80
Hi  75 184
Hi  76 193
Hi  77 159
Hi  78 86
Hi  79 112
Hi  80 108
Hi  81 67
Hi  82 148
Hi  83 105
Hi  84 15
Hi  85 138
Hi  86 175
Hi  87 155
Hi  88 23
Hi  89 108
Hi  90 124
Hi  91 17
Hi  92 67
Hi  93 20
Hi  94 91
Hi  95 33
Hi  96 27
Hi  97 3

In [224]:
predictions, targets = train['pred_word'], train['orig']

In [225]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

accuracy = accuracy_score(predictions, targets)
recall = recall_score(predictions, targets, average='macro')
f1 = f1_score(predictions, targets, average='macro')

In [226]:
f1

0.00863043639327258

In [227]:
recall

0.009111654441727792

In [228]:
accuracy

0.01758641600970285